### requirements for the following codings


In [1]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 996 kB/s 


### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [2]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings

In [4]:
import numpy as np
import pandas as pd

In [7]:
# training dataset loading
dataset = pd.read_excel('TTCA_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('TTCA_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label'] 
y_train = np.array(y_train) # transformed as np.array for CNN model

In [6]:
# test dataset loading
dataset = pd.read_excel('TTCA_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('TTCA_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [8]:
# assign the dataset 
X_train_data_name = 'TTCA_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'TTCA_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [9]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(788, 320)
(197, 320)
(788,)
(197,)


### Model architecture

In [10]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [11]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
89/89 [==============================] - 2s 12ms/step - loss: 1.1575 - accuracy: 0.5853 - val_loss: 0.6937 - val_accuracy: 0.5570 - lr: 0.1000
Epoch 2/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6668 - accuracy: 0.6121 - val_loss: 0.7071 - val_accuracy: 0.5570 - lr: 0.1000
Epoch 3/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6443 - accuracy: 0.6432 - val_loss: 0.6868 - val_accuracy: 0.5823 - lr: 0.0600
Epoch 4/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6297 - accuracy: 0.6827 - val_loss: 0.6905 - val_accuracy: 0.5696 - lr: 0.0600
Epoch 5/200
89/89 [==============================] - 1s 14ms/step - loss: 0.6182 - accuracy: 0.6756 - val_loss: 0.6475 - val_accuracy: 0.6076 - lr: 0.0600
Epoch 6/200
89/89 [==============================] - 2s 18ms/step - loss: 0.5747 - accuracy: 0.7080 - val_loss: 0.6222 - val_accuracy: 0.6582 - lr: 0.0360
Epoch 7/200
89/89 [==============================] - 2s 18ms/step - loss:

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


89/89 [==============================] - 2s 11ms/step - loss: 1.4166 - accuracy: 0.5952 - val_loss: 0.7043 - val_accuracy: 0.4937 - lr: 0.1000
Epoch 2/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6626 - accuracy: 0.6192 - val_loss: 0.7214 - val_accuracy: 0.4937 - lr: 0.1000
Epoch 3/200
89/89 [==============================] - 1s 10ms/step - loss: 0.6314 - accuracy: 0.6474 - val_loss: 0.7065 - val_accuracy: 0.5316 - lr: 0.0600
Epoch 4/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6119 - accuracy: 0.6601 - val_loss: 0.6638 - val_accuracy: 0.5823 - lr: 0.0600
Epoch 5/200
89/89 [==============================] - 1s 9ms/step - loss: 0.5798 - accuracy: 0.6812 - val_loss: 0.6671 - val_accuracy: 0.6076 - lr: 0.0600
Epoch 6/200
89/89 [==============================] - 1s 9ms/step - loss: 0.5524 - accuracy: 0.7193 - val_loss: 0.6187 - val_accuracy: 0.6582 - lr: 0.0360
Epoch 7/200
89/89 [==============================] - 1s 9ms/step - loss: 0.5355 - accu

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

89/89 [==============================] - 2s 11ms/step - loss: 1.7239 - accuracy: 0.5628 - val_loss: 0.6737 - val_accuracy: 0.6709 - lr: 0.1000
Epoch 2/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6623 - accuracy: 0.6164 - val_loss: 0.6446 - val_accuracy: 0.6709 - lr: 0.1000
Epoch 3/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6168 - accuracy: 0.6559 - val_loss: 0.6243 - val_accuracy: 0.6709 - lr: 0.0600
Epoch 4/200
89/89 [==============================] - 1s 9ms/step - loss: 0.5873 - accuracy: 0.6869 - val_loss: 0.6027 - val_accuracy: 0.7089 - lr: 0.0600
Epoch 5/200
89/89 [==============================] - 1s 9ms/step - loss: 0.5474 - accuracy: 0.7292 - val_loss: 0.5708 - val_accuracy: 0.6709 - lr: 0.0600
Epoch 6/200
89/89 [==============================] - 1s 9ms/step - loss: 0.5055 - accuracy: 0.7532 - val_loss: 0.5915 - val_accuracy: 0.7215 - lr: 0.0360
Epoch 7/200
89/89 [==============================] - 1s 9ms/step - loss: 0.4782 - accur

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


89/89 [==============================] - 2s 11ms/step - loss: 0.9479 - accuracy: 0.5915 - val_loss: 0.6610 - val_accuracy: 0.6282 - lr: 0.1000
Epoch 2/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6765 - accuracy: 0.6028 - val_loss: 0.6623 - val_accuracy: 0.6282 - lr: 0.1000
Epoch 3/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6627 - accuracy: 0.6183 - val_loss: 0.6562 - val_accuracy: 0.6282 - lr: 0.0600
Epoch 4/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6437 - accuracy: 0.6423 - val_loss: 0.6505 - val_accuracy: 0.6282 - lr: 0.0600
Epoch 5/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6236 - accuracy: 0.6676 - val_loss: 0.6455 - val_accuracy: 0.6538 - lr: 0.0600
Epoch 6/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6145 - accuracy: 0.6662 - val_loss: 0.6273 - val_accuracy: 0.6538 - lr: 0.0360
Epoch 7/200
89/89 [==============================] - 1s 10ms/step - loss: 0.6001 - accu

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


89/89 [==============================] - 2s 11ms/step - loss: 0.9642 - accuracy: 0.5662 - val_loss: 0.6571 - val_accuracy: 0.6923 - lr: 0.1000
Epoch 2/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6834 - accuracy: 0.5859 - val_loss: 0.6269 - val_accuracy: 0.6923 - lr: 0.1000
Epoch 3/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6806 - accuracy: 0.5859 - val_loss: 0.6351 - val_accuracy: 0.6923 - lr: 0.0600
Epoch 4/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6779 - accuracy: 0.5901 - val_loss: 0.6170 - val_accuracy: 0.6923 - lr: 0.0600
Epoch 5/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6601 - accuracy: 0.6169 - val_loss: 0.6284 - val_accuracy: 0.6923 - lr: 0.0600
Epoch 6/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6512 - accuracy: 0.6239 - val_loss: 0.6171 - val_accuracy: 0.7179 - lr: 0.0360
Epoch 7/200
89/89 [==============================] - 1s 9ms/step - loss: 0.6369 - accur

In [12]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.7285783836416748 ± 0.04780021761134589
0.7487234101926231 ± 0.05132703673475653
0.7176646108507146 ± 0.06208603593870916
0.7797822095345315 ± 0.08959138605474613
0.4236075475711559 ± 0.07562842881460459
0.7156243740189678 ± 0.03963673957508091


### model evaluation in test dataset

In [13]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


99/99 [==============================] - 2s 11ms/step - loss: 1.1373 - accuracy: 0.5952 - val_loss: 0.6711 - val_accuracy: 0.6193 - lr: 0.1000
Epoch 2/200
99/99 [==============================] - 1s 9ms/step - loss: 0.6507 - accuracy: 0.6332 - val_loss: 0.6661 - val_accuracy: 0.6193 - lr: 0.1000
Epoch 3/200
99/99 [==============================] - 1s 9ms/step - loss: 0.6132 - accuracy: 0.6789 - val_loss: 0.6564 - val_accuracy: 0.6193 - lr: 0.0600
Epoch 4/200
99/99 [==============================] - 1s 9ms/step - loss: 0.5952 - accuracy: 0.6789 - val_loss: 0.6485 - val_accuracy: 0.6345 - lr: 0.0600
Epoch 5/200
99/99 [==============================] - 1s 10ms/step - loss: 0.5879 - accuracy: 0.7018 - val_loss: 0.6057 - val_accuracy: 0.6701 - lr: 0.0600
Epoch 6/200
99/99 [==============================] - 1s 9ms/step - loss: 0.5399 - accuracy: 0.7234 - val_loss: 0.5417 - val_accuracy: 0.7259 - lr: 0.0360
Epoch 7/200
99/99 [==============================] - 1s 9ms/step - loss: 0.5055 - accu

In [14]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.751269035532995
0.7433639295460184
0.762589928057554
0.7241379310344828
0.45687464281069984
0.7882513661202186


In [15]:
model.save('TTCA_tensorflow_model',save_format = 'tf') 
!zip -r /content/TTCA_tensorflow_model.zip /content/TTCA_tensorflow_model

  adding: content/TTCA_tensorflow_model/ (stored 0%)
  adding: content/TTCA_tensorflow_model/variables/ (stored 0%)
  adding: content/TTCA_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/TTCA_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 39%)
  adding: content/TTCA_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/TTCA_tensorflow_model/assets/ (stored 0%)
  adding: content/TTCA_tensorflow_model/keras_metadata.pb (deflated 89%)
